In [6]:
import pandas as pd
from pathlib import Path
import numpy as np
import json
import requests
from requests.auth import HTTPBasicAuth
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import plotly.express as px

In [7]:
# Setup Pandas
pd.set_option('display.max_columns', None)

In [8]:
# Set up Teams
with Path(Path.home(), '.aircall', 'teams.json').open() as team: 
    teams = json.load(team)

In [9]:
# get the API Key
with Path(Path.home(), ".aircall", "aircall_credentials.json").open() as f:
    apicreds = json.load(f)

In [10]:
# Setup Aircall API
api_id = apicreds['api_key']['api_id']
api_token = apicreds['api_key']['api_token']
aircall_base_url = "https://api.aircall.io/v1"
ac_auth = HTTPBasicAuth(api_id, api_token)

In [11]:
# Make a test call
response = requests.get("https://api.aircall.io/v1/ping", auth=ac_auth)
if response.status_code == 200: print("Response: 200 ✅")

Response: 200 ✅


# Get specific calls that match a given filter

In [12]:
# Setup the time window

def time_window(start_time=datetime.today() - timedelta(days=1)):
    dt = start_time.replace(hour=7, minute=0, second=0, microsecond=0)
    start_period_datetime = dt
    end_period_datetime = start_period_datetime + relativedelta(months=1)
    if (end_period_datetime) > datetime.today():
        end_period_datetime = datetime.today()
    start_period_unix = pd.Timestamp(start_period_datetime).timestamp()
    end_period_unix = pd.Timestamp(end_period_datetime).timestamp()

    window = {
        'start_period_datetime' : start_period_datetime, 
        'end_period_datetime' : end_period_datetime, 
        'start_period_unix' : start_period_unix,
        'end_period_unix' : end_period_unix        
    }

    return window

In [13]:
def search_call(starttime=(datetime.today() - timedelta(days=1)).timestamp(), endtime=datetime.today().timestamp(), per_page=50, page=1):
    calls_endpoint_addr = aircall_base_url + '/calls/search'

    # Define the parameters
    params = {
        'from': starttime,
        'to': endtime,
        'per_page': per_page,
        'page' : page
        }

    # Call the endpoint
    get_calls = requests.get(calls_endpoint_addr, auth=ac_auth, params=params)
    return json.loads(get_calls.text)

In [14]:
# format calls in a pandas DataFrame

def create_calls_dataframe(calls_request=[{'started_at':0,'answered_at':0, 'ended_at':0}]):
    # Create DataFrame
    calls_df = pd.DataFrame(calls_request)
    
    # Format date columns
    if 'started_at' in calls_df:
        calls_df['call_start_at'] = pd.to_datetime(calls_df['started_at'], unit='s')
    if 'answered_at' in calls_df:
        calls_df['call_ansered_at'] = pd.to_datetime(calls_df['answered_at'], unit='s')
    if 'ended_at' in calls_df:
        calls_df['call_end_at'] = pd.to_datetime(calls_df['ended_at'], unit='s')

    return calls_df

In [15]:
# Get all calls in a loop
def search_calls_dataframe(starttime=(datetime.today() - timedelta(days=1)).timestamp(), endtime=datetime.today().timestamp(), per_page=50, pages=2):
    per_page = 50
    page = 1
    calls_list = []
    meta_list = []

    # Get the total number of elements in the request
    totals = calls_json = search_call(starttime=starttime, per_page=1)['meta']['total']
    total_items = int(totals)

    # Calculate how many pages have to be retrieved and how much rest
    pages_to_retrieve = int(total_items/per_page)
    items_left = total_items - (pages_to_retrieve * per_page)
    print(f'Total items: {total_items}. Pages to retrieve: {pages_to_retrieve}. Rest Items: {items_left}')

    # Bulk-Loop over all Pages from the start-point
    for i in range(pages_to_retrieve):
        print(f'Pages to retrieve: {pages_to_retrieve}. Current page: {page}')
        calls_json = search_call(starttime=starttime, endtime=endtime, per_page=per_page, page=page)
        if 'calls' in calls_json:
            calls_list.extend(calls_json['calls'])
            print(f'last item in list: {calls_list[-1]["started_at"]}')
        page = page + 1
        if page >= pages_to_retrieve + 1:
            break
    # Calculate the Calls per page for the last three requests
    items_left_bisected = int(items_left/2)
    items_rest = items_left - items_left_bisected*2

    ## Get the rest of the items on the last page
    calls_json = search_call(starttime=starttime, endtime=endtime, per_page=items_left, page=page)
    if 'calls' in calls_json:
        calls_list.extend(calls_json['calls'])
            
    print(f'end of page. Pages to retrieve: {pages_to_retrieve}. Current page: {page}')
    
    return create_calls_dataframe(calls_list)
    

## Request the calls in the given timeframe

In [20]:
starttime = datetime(2024, 7, 1, 7, 00).timestamp()
endtime = datetime(2024, 7, 31, 23, 00).timestamp()
# calls_list = search_calls_dataframe(starttime=starttime, endtime=endtime)

In [22]:
calls_list.shape

(1004, 31)

# Preparation for Statistics

In [25]:
## Drop colums that are not needed and expand json in columns
all_calls_df = calls_list
calls_list['teams_expanded'] = pd.json_normalize(calls_list.teams)
calls_cleaned_user_expand1 = pd.json_normalize(calls_list.user)
calls_cleaned_user_expand = calls_cleaned_user_expand1.rename(columns={"name": "name_user", "id":"id_user", "email": "email_user", 
                                                                       "available":"available_user", "availability_status":"availability_status_user", 
                                                                       "created_at":"created_at_user", "time_zone":"time_zone_user", 
                                                                       "language":"language_user","wrap_up_time":"wrap_up_time_user"})
calls_cleaned_team_expand1 = pd.json_normalize(calls_list.teams_expanded)
calls_cleaned_team_expand = calls_cleaned_team_expand1.rename(columns={"name": "name_team", "id":"id_team", "direct_link":"direct_link_team",
                                                                      "created_at":"created_at_team"})
calls_cleaned_team_expand['id_team'] = calls_cleaned_team_expand.id_team.astype('Int64')

calls_cleaned_number_expand1 = pd.json_normalize(calls_list.number)
calls_cleaned_number_expand = calls_cleaned_number_expand1.rename(columns={"id":"id_num", "direct_link":"direct_link_num", "name":"name_num", "digits":"digits_num", 
                                                                           "country":"country_num", "time_zone":"time_zone_num","open":"open_num",
                                                                           "availability_status":"availability_status_num", "is_ivr":"is_ivr_num", 
                                                                           "live_recording_activated":"live_recording_activated_num", "priority":"priority_num",
                                                                           "created":"created_at_num", "messages.welcome":"messages.welcome_num", 
                                                                           "messages.waiting":"messages.waiting_num","messages.ivr":"messages.ivr_num",
                                                                           "messages.voicemail":"messages.voicemail_num","messages.closed":"messages.closed_num",
                                                                           "messages.callback_later":"messages.callback_later_num","messages.unanswered_call":"messages.unanswered_call_num",
                                                                           "messages.after_hours":"messages.after_hours_num", "messages.ringing_tone":"messages.ringing_tone_num"})


## Join all Tables
all_calls_new = all_calls_df.join(calls_cleaned_user_expand, lsuffix='_calls', rsuffix='_users')
all_calls_new_joined_1 = all_calls_new.join(calls_cleaned_team_expand, lsuffix='_users', rsuffix='_teams')
all_calls_new_joined = all_calls_new_joined_1.join(calls_cleaned_number_expand, lsuffix='_original', rsuffix='_number')


In [26]:
# Get the columns for datetimes
all_calls_new_joined['start_dt'] = pd.to_datetime(all_calls_new_joined['started_at'].astype(int,errors='ignore'),unit='s')
all_calls_new_joined['answer_dt'] = pd.to_datetime(all_calls_new_joined['answered_at'].astype(int,errors='ignore'),unit='s')
all_calls_new_joined['end_dt'] = pd.to_datetime(all_calls_new_joined['ended_at'].astype(int,errors='ignore'),unit='s')

In [27]:
## Get the columns for days
all_calls_new_joined['start_day'] = all_calls_new_joined['start_dt'].dt.date
all_calls_new_joined['answer_day'] = all_calls_new_joined['answer_dt'].dt.date
all_calls_new_joined['end_day'] = all_calls_new_joined['end_dt'].dt.date

In [28]:
## Get the columns for hours
all_calls_new_joined['start_hour'] = all_calls_new_joined['start_dt'].dt.hour
all_calls_new_joined['answer_hour'] = all_calls_new_joined['answer_dt'].dt.hour
all_calls_new_joined['end_hour'] = all_calls_new_joined['end_dt'].dt.hour
all_calls_new_joined['duration_minutes'] = all_calls_new_joined['duration'].apply(lambda x: np.round(x/60,2))

In [29]:
all_calls_new_joined['start_weekday'] = all_calls_new_joined['start_dt'].dt.day_name()
all_calls_new_joined['answer_weekday'] = all_calls_new_joined['answer_dt'].dt.day_name()
all_calls_new_joined['end_weekday'] = all_calls_new_joined['end_dt'].dt.day_name()

In [30]:
# Get the columns for calendar week
all_calls_new_joined['start_calendarweek'] = all_calls_new_joined['start_dt'].dt.isocalendar().week
all_calls_new_joined['answer_calendarweek'] = all_calls_new_joined['answer_dt'].dt.isocalendar().week
all_calls_new_joined['end_calendarweek'] = all_calls_new_joined['end_dt'].dt.isocalendar().week

In [31]:
## Create final list for statistics

calls = all_calls_new_joined[['id','id_num','name_num','digits_num','status','direction','call_start_at','call_ansered_at','call_end_at',
                              'duration','missed_call_reason','id_team','name_team','raw_digits','user','contact','archived','assigned_to','tags',
                              'transferred_by','transferred_to','comments','country_code_a2','id_user','direct_link_users','name_user','email_user',
                              'available_user','availability_status_user','created_at_user','time_zone_user','language_user','wrap_up_time_user',
                              'country_num','time_zone_num','open_num','availability_status_num','is_ivr_num','live_recording_activated_num',
                              'priority_num','created_at','start_dt','answer_dt','end_dt','start_day','answer_day','end_day','start_hour',
                              'answer_hour','end_hour','duration_minutes', 'start_weekday', 'answer_weekday','end_weekday','start_calendarweek','answer_calendarweek','end_calendarweek']]

In [32]:
## That whole thing as a function

def cleanupCallsDF(calls_table = []):
    calls_list = calls_table
    
    if 'teams_expanded' in calls_list:
        calls_list['teams_expanded'] = pd.json_normalize(calls_list.teams)
        calls_cleaned_user_expand1 = pd.json_normalize(calls_list.user)
        calls_cleaned_user_expand = calls_cleaned_user_expand1.rename(columns={"name": "name_user", "id":"id_user", "email": "email_user", 
                                                                               "available":"available_user", "availability_status":"availability_status_user", 
                                                                               "created_at":"created_at_user", "time_zone":"time_zone_user", 
                                                                               "language":"language_user","wrap_up_time":"wrap_up_time_user"})
        calls_cleaned_team_expand1 = pd.json_normalize(calls_list.teams_expanded)
        calls_cleaned_team_expand = calls_cleaned_team_expand1.rename(columns={"name": "name_team", "id":"id_team", "direct_link":"direct_link_team",
                                                                              "created_at":"created_at_team"})
        calls_cleaned_team_expand['id_team'] = calls_cleaned_team_expand.id_team.astype('Int64')
        
        calls_cleaned_number_expand1 = pd.json_normalize(calls_list.number)
        calls_cleaned_number_expand = calls_cleaned_number_expand1.rename(columns={"id":"id_num", "direct_link":"direct_link_num", "name":"name_num", "digits":"digits_num", 
                                                                                   "country":"country_num", "time_zone":"time_zone_num","open":"open_num",
                                                                                   "availability_status":"availability_status_num", "is_ivr":"is_ivr_num", 
                                                                                   "live_recording_activated":"live_recording_activated_num", "priority":"priority_num",
                                                                                   "created":"created_at_num", "messages.welcome":"messages.welcome_num", 
                                                                                   "messages.waiting":"messages.waiting_num","messages.ivr":"messages.ivr_num",
                                                                                   "messages.voicemail":"messages.voicemail_num","messages.closed":"messages.closed_num",
                                                                                   "messages.callback_later":"messages.callback_later_num","messages.unanswered_call":"messages.unanswered_call_num",
                                                                                   "messages.after_hours":"messages.after_hours_num", "messages.ringing_tone":"messages.ringing_tone_num"})
        
    
        ## Join all Tables
        all_calls_new = all_calls_df.join(calls_cleaned_user_expand, lsuffix='_calls', rsuffix='_users')
        all_calls_new_joined_1 = all_calls_new.join(calls_cleaned_team_expand, lsuffix='_users', rsuffix='_teams')
        all_calls_new_joined = all_calls_new_joined_1.join(calls_cleaned_number_expand, lsuffix='_original', rsuffix='_number')
    
    return ''

# Hotlines

In [33]:
### Configure the table for holtines
hotlines_df = calls[calls['name_num'].isin(teams['hotlines'])]

### How many calls does each hotline receive?

In [34]:
piv_hotline_calls = pd.pivot_table(hotlines_df, index='name_num', columns='direction', values='id', aggfunc='count', margins=True)
print(f'Calls per hotline between{datetime.fromtimestamp(starttime)} and {datetime.fromtimestamp(endtime)}')
# piv_hotline_calls.fillna('-')

Calls per hotline between2024-07-01 07:00:00 and 2024-07-31 23:00:00


In [35]:
only_support = hotlines_df[hotlines_df['name_num']=='DE Tech Support']
# only_support[only_support['direction']=='outbound']

### During wich hour do we receive the most calls?

In [36]:
print(f'Incoming Calls (per hour) between {datetime.fromtimestamp(starttime)} and {datetime.fromtimestamp(endtime)}')
calls_per_hours = pd.pivot_table(hotlines_df, index='start_hour', values='duration_minutes', aggfunc='count', margins=True)
# calls_per_hours

Incoming Calls (per hour) between 2024-07-01 07:00:00 and 2024-07-31 23:00:00


## How many calls per weekday?

In [37]:
## Condensed view:

print(f'Weekly Calls per Day of the Week {datetime.fromtimestamp(starttime)} and {datetime.fromtimestamp(endtime)}')
calls_per_weekday = pd.pivot_table(hotlines_df[hotlines_df['name_num'].isin(['EN Tech Support', 'DE Tech Support', 'IVR Deutsch', 'DE Emergency', 'EN Emergency'])], index=['start_weekday','name_num'], values='id', aggfunc='count', margins=True)
calls_per_weekday

Weekly Calls per Day of the Week 2024-07-01 07:00:00 and 2024-07-31 23:00:00


## Weekly Calls per Calendarweek (KW)

In [38]:
## Number of calls per Calendarweek
print(f'Weekly Calls per Calendarweek (KW) {datetime.fromtimestamp(starttime)} and {datetime.fromtimestamp(endtime)}')
weekly_calls_per_hotline = pd.pivot_table(hotlines_df[hotlines_df['name_num'].isin(['EN Tech Support', 'DE Tech Support', 'IVR Deutsch', 'DE Emergency', 'EN Emergency'])], index=['name_num','start_calendarweek'], values='id', aggfunc='count', margins=True)
weekly_calls_per_hotline

Weekly Calls per Calendarweek (KW) 2024-07-01 07:00:00 and 2024-07-31 23:00:00


### How many calls per weekday (All Hotlines)?

In [39]:
# print(f'Weekly Calls per Day of the Week {datetime.fromtimestamp(starttime)} and {datetime.fromtimestamp(endtime)}')
# pd.pivot_table(hotlines_df, index=['start_weekday','name_num'], values='id', aggfunc='count', margins=True)

### Average duration of calls on the support hotline

In [40]:
inbound_avg = np.round(calls[(calls['name_num'] == 'DE Tech Support') &  (calls['direction'] == 'inbound')]['duration'].mean()/60, 2)
outbound_avg = np.round(calls[(calls['name_num'] == 'DE Tech Support') &  (calls['direction'] == 'outbound')]['duration'].mean()/60, 2)
# print(f'Average duration of inbound calls: {inbound_avg} minutes \nAverage duration of outbound calls: {outbound_avg} minutes \nTimeframe between {datetime.fromtimestamp(starttime)} and {datetime.fromtimestamp(endtime)}')


d = {'Average duration of calls': ['Inbound calls', 'Outbound calls'], 'Time in minutes': [inbound_avg,outbound_avg]}
avg_duration_of_calls = pd.DataFrame(d)
avg_duration_of_calls

### Missed Calls

In [42]:
piv_call_missed = pd.pivot_table(calls, index='start_day', columns='missed_call_reason', values='id', aggfunc='count', margins=True)
print(f'Missed calls between{datetime.fromtimestamp(starttime)} and {datetime.fromtimestamp(endtime)}')
piv_call_missed

Missed calls between2024-07-01 07:00:00 and 2024-07-31 23:00:00


### Missed calls per team-hotline

In [43]:
## Pivot Table 
piv_missed_team = pd.pivot_table(hotlines_df, index='name_num', columns='missed_call_reason', values='id', aggfunc='count', margins=True)
print(f'Missed Calls per Hotline Number between {datetime.fromtimestamp(starttime)} and {datetime.fromtimestamp(endtime)}')
piv_missed_by_team = piv_missed_team.fillna('-')
piv_missed_by_team

Missed Calls per Hotline Number between 2024-07-01 07:00:00 and 2024-07-31 23:00:00


### List of missed calls on the support-hotline

In [44]:
support_missed_df = hotlines_df[(hotlines_df['name_num'] == 'DE Tech Support') & (hotlines_df['missed_call_reason'] =='agents_did_not_answer')]
df_support_calls_missed = support_missed_df[['id','name_num','call_start_at','call_ansered_at','call_end_at','raw_digits','country_code_a2','start_day','duration_minutes', 'start_hour']]
df_support_calls_missed

## Missed Calls during which hour?

In [45]:
# during which hours do we miss most of the calls in support?
print(f'Calls per Hour of the Day between {datetime.fromtimestamp(starttime)} and {datetime.fromtimestamp(endtime)}')
df_missed_calls_hour = pd.pivot_table(support_missed_df, index='start_hour', values='duration_minutes', aggfunc='count', margins=True)
df_missed_calls_hour

Calls per Hour of the Day between 2024-07-01 07:00:00 and 2024-07-31 23:00:00


In [46]:
# Summarize all data in an Excel Sheet

def create_excel_output(): 
    ## Create Summary Table
    summary_dict = {
        'sheet name': ['number_of_hotline_calls',
                       'support_outbound_calls',
                       'calls_per_hours',
                       'calls_per_weekday',
                       'weekly_calls_per_hotline',
                       'avg_duration_of_calls',
                       'piv_call_missed',
                       'calls_missed_by_team',
                       'df_support_calls_missed',
                       'df_missed_calls_hour'                   
                      ],
        'topic':['How many calls does each hotline receive?',
                 'List of outbound calls on the support-hotline',
                 'During wich hour do we receive the most calls?',
                 'How many calls per weekday?',
                 'Weekly Calls per Calendarweek (KW)',
                'Average duration of calls on the support hotline',
                 f'Missed Calls between {datetime.fromtimestamp(starttime)} and {datetime.fromtimestamp(endtime)}',
                 f'Missed Calls per Hotline Number between {datetime.fromtimestamp(starttime)} and {datetime.fromtimestamp(endtime)}',
                 'List of missed calls on the support-hotline',
                'Missed Calls during which hour?' 
                ]
    }
    summary_df = pd.DataFrame(data=summary_dict)
    
    
    ## Create Excel Sheet
    
    with pd.ExcelWriter('output.xlsx') as writer:  
        summary_df.to_excel(writer, sheet_name='Summary')
        piv_hotline_calls.fillna('-').to_excel(writer, sheet_name='number_of_hotline_calls')
        only_support[only_support['direction']=='outbound'].to_excel(writer, sheet_name='support_outbound_calls')
        calls_per_hours.to_excel(writer, sheet_name='calls_per_hour')
        calls_per_weekday.to_excel(writer, sheet_name='calls_per_weekday')
        weekly_calls_per_hotline.to_excel(writer, sheet_name='weekly_calls_per_hotline')
        avg_duration_of_calls.to_excel(writer, sheet_name='avg_duration_of_calls')
        piv_call_missed.to_excel(writer, sheet_name='all_calls_missed')
        piv_missed_by_team.to_excel(writer, sheet_name='all_calls_missed')
        df_support_calls_missed.to_excel(writer, sheet_name='calls_missed_by_team')
        df_missed_calls_hour.to_excel(writer, sheet_name='missed_calls_per_hour')

In [47]:
# create_excel_output()

# Finished